In [16]:
# Get Dataset
!git clone https://github.com/thatbrguy/Dropbox-Uploader.git
%cd Dropbox-Uploader
!echo "WCOL1o0mlMAAAAAAAAAUkI3btdYWm3qFFxOW5cB_dlGVt1DGoFHFNNpYNOGud2Xt" > token.txt
!bash dropbox_uploader.sh

Cloning into 'Dropbox-Uploader'...
remote: Enumerating objects: 951, done.
remote: Total 951 (delta 0), reused 0 (delta 0), pack-reused 951
Receiving objects: 100% (951/951), 309.20 KiB | 3.12 MiB/s, done.
Resolving deltas: 100% (505/505), done.
/content/Dropbox-Uploader/Dropbox-Uploader
Dropbox Uploader v1.0
Andrea Fabrizi - andrea.fabrizi@gmail.com

Usage: dropbox_uploader.sh [PARAMETERS] COMMAND...

Commands:
	 upload   <LOCAL_FILE/DIR ...>  <REMOTE_FILE/DIR>
	 download <REMOTE_FILE/DIR> [LOCAL_FILE/DIR]
	 delete   <REMOTE_FILE/DIR>
	 move     <REMOTE_FILE/DIR> <REMOTE_FILE/DIR>
	 copy     <REMOTE_FILE/DIR> <REMOTE_FILE/DIR>
	 mkdir    <REMOTE_DIR>
	 list     [REMOTE_DIR]
	 monitor  [REMOTE_DIR] [TIMEOUT]
	 share    <REMOTE_FILE>
	 saveurl  <URL> <REMOTE_DIR>
	 search   <QUERY>
	 info
	 space
	 unlink

Optional parameters:
	-f <FILENAME> Load the configuration file from a specific file
	-s            Skip already existing files when download/upload. Default: Overwrite
	-d           

In [18]:
! ./dropbox_uploader.sh download dataset.zip

 > Downloading "/dataset.zip" to "/content/Dropbox-Uploader/Dropbox-Uploader/dataset.zip"... DONE


In [19]:
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from keras.layers import *
from keras.models import *
from keras import optimizers
import keras
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
import os
!pip install Zipfile36
import zipfile

In [20]:
#Extract dataset
local_zip='./dataset.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('./dataset')
zip_ref.close()
%cd /content/Dropbox-Uploader/dataset

/content/Dropbox-Uploader/dataset


In [0]:
# Read data
dog=['bullmastiff', 'chowchow', 'pug', 'maltese', 'huskysibir', 'dachshund', 'dalmatian', 'corgi', 'chihuahua', 'yorkshire']
X_train=[]
y_train=[]
for index in range(len(dog)):
  for i in os.listdir('./'+dog[index]):
    X_train.append(cv2.resize(cv2.imread('./'+dog[index]+'/'+i),(224,224)))
    y_train.append(index)

In [0]:
# Process data
X_train,X_val,y_train,y_val=train_test_split(X_train,y_train,test_size=0.2,random_state=42)
y_train=keras.utils.to_categorical(y_train,10)
y_val=keras.utils.to_categorical(y_val,10)
X_train=np.array(X_train)
X_val=np.array(X_val)
y_train=np.array(y_train)
y_val=np.array(y_val)
X_train=X_train.astype('float16')
X_val=X_val.astype('float16')
X_train=X_train/255
X_val=X_val/255

In [0]:
base_model = VGG16(weights='imagenet', input_shape=(224,224,3),include_top=False)
base_model.summary()

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
bloc

In [0]:
add_model = Sequential()
add_model.add(Flatten(input_shape=base_model.output_shape[1:]))
add_model.add(Dense(1024, activation='relu'))
add_model.add(Dense(256, activation='relu'))
add_model.add(Dense(10, activation='softmax'))
model = Model(inputs=base_model.input, outputs=add_model(base_model.output))
model.compile(loss='categorical_crossentropy', optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [0]:
model.fit(X_train, y_train, batch_size=32,
              nb_epoch=1,validation_data=(X_val, y_val), shuffle=True)

Instructions for updating:
Use tf.cast instead.


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  


Train on 9708 samples, validate on 2427 samples
Epoch 1/1
9708/9708 [==============================] - 295s 30ms/step - loss: 1.1988 - acc: 0.5876 - val_loss: 0.5439 - val_acc: 0.8179


In [0]:
model.fit(X_train, y_train, batch_size=32,
              nb_epoch=2,validation_data=(X_val, y_val), shuffle=True)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  


Train on 9708 samples, validate on 2427 samples
Epoch 1/2
9708/9708 [==============================] - 280s 29ms/step - loss: 0.3985 - acc: 0.8678 - val_loss: 0.4112 - val_acc: 0.8628
Epoch 2/2
9708/9708 [==============================] - 281s 29ms/step - loss: 0.2387 - acc: 0.9227 - val_loss: 0.4109 - val_acc: 0.8719


In [0]:
model.fit(X_train, y_train, batch_size=32,
              nb_epoch=2,validation_data=(X_val, y_val), shuffle=True)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  


Train on 9708 samples, validate on 2427 samples
Epoch 1/2
9708/9708 [==============================] - 282s 29ms/step - loss: 0.1461 - acc: 0.9549 - val_loss: 0.3747 - val_acc: 0.8801
Epoch 2/2
9708/9708 [==============================] - 282s 29ms/step - loss: 0.0759 - acc: 0.9804 - val_loss: 0.3976 - val_acc: 0.8859


In [32]:
res=model.predict(X_train,verbose=1)

9708/9708 [==============================] - 91s 9ms/step
0.0


In [36]:
# Top 1 train accuracy
dem=0
for i in range(res.shape[0]):
  if (np.argmax(res[i])==np.argmax(y_train[i])): dem=dem+1
print(dem/res.shape[0])


0.9957766790276061


In [39]:
# Top 3 train accuracy
dem=0
for i in range(res.shape[0]):
  if (np.argmax(y_train[i]) in res[i].argsort()[-3:]): dem=dem+1
print(dem/res.shape[0])

0.999896992171405


In [40]:
res=model.predict(X_val,verbose=1)
# Top 1 validation accuracy
dem=0
for i in range(res.shape[0]):
  if (np.argmax(res[i])==np.argmax(y_val[i])): dem=dem+1
print(dem/res.shape[0])

2427/2427 [==============================] - 23s 9ms/step
0.8858673259167696


In [41]:
# Top 3 validation accuracy
dem=0
for i in range(res.shape[0]):
  if (np.argmax(y_val[i]) in res[i].argsort()[-3:]): dem=dem+1
print(dem/res.shape[0])

0.9715698393077874
